In [0]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS healthcare_bronze
LOCATION 'abfss://bronze@mystoacc.dfs.core.windows.net/'
""")

spark.sql("""
CREATE DATABASE IF NOT EXISTS healthcare_silver
LOCATION 'abfss://silver@mystoacc.dfs.core.windows.net/'
""")

spark.sql("""
CREATE DATABASE IF NOT EXISTS healthcare_gold
LOCATION 'abfss://gold@mystoacc.dfs.core.windows.net/'
""")

print("Databases criados!")

In [0]:
spark.sql("USE healthcare_bronze")

# Dimensões
dimensions = [
    ("dim_data", "dim_data"),
    ("dim_clinica", "dim_clinica"),
    ("dim_medico", "dim_medico"),
    ("dim_diagnostico", "dim_diagnostico"),
    ("dim_exame", "dim_exame"),
    ("dim_paciente", "dim_paciente")
]

print("Criando dimensoes...")
for table_name, path in dimensions:
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {table_name}
    USING DELTA
    LOCATION 'abfss://bronze@mystoacc.dfs.core.windows.net/{path}'
    """)
    print(f"Tabela {table_name} criada")

# Fato
print("\nCriando fato...")
spark.sql("""
CREATE TABLE IF NOT EXISTS fato_consultas
USING DELTA
LOCATION 'abfss://bronze@mystoacc.dfs.core.windows.net/facts/fato_consultas'
""")
print("Tabela fato_consultas criada")

# PII
print("\nCriando tabelas PII...")
spark.sql("""
CREATE TABLE IF NOT EXISTS pii_paciente_identidade
USING DELTA
LOCATION 'abfss://bronze@mystoacc.dfs.core.windows.net/pii/paciente_identidade'
""")
print("Tabela pii_paciente_identidade criada")

spark.sql("""
CREATE TABLE IF NOT EXISTS pii_medico_identidade
USING DELTA
LOCATION 'abfss://bronze@mystoacc.dfs.core.windows.net/pii/medico_identidade'
""")
print("Tabela pii_medico_identidade criada")

print("\nTodas as tabelas foram criadas com sucesso")

# Validar
display(spark.sql("SHOW TABLES IN healthcare_bronze"))

In [0]:
%sql
select * from healthcare_bronze.fato_consultas